In [1]:
# Stock Price Tracker
import requests
from bs4 import BeautifulSoup
import smtplib
import time
import csv
import datetime

In [ ]:
def get_stock_price(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}"
    headers = {"User-Agent": "Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    price = soup.find('fin-streamer', {'data-symbol': ticker}).text
    return float(price.replace(',', ''))

def log_price(ticker, price):
    today = datetime.date.today()
    with open(f'{ticker}_prices.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([today, price])

def send_alert(ticker, price, threshold, direction):
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.login('your_email@gmail.com', 'app_password')
    subject = f"{ticker} Alert: Price {'above' if direction == 'up' else 'below'} ${threshold}!"
    body = f"Current {ticker} price: ${price}"
    server.sendmail('your_email@gmail.com', 'recipient@email.com', f"Subject: {subject}\n\n{body}")
    server.quit()

# Main loop
ticker = "AAPL"
upper_threshold = 180
lower_threshold = 160

while True:
    price = get_stock_price(ticker)
    log_price(ticker, price)
    
    if price >= upper_threshold:
        send_alert(ticker, price, upper_threshold, 'up')
    elif price <= lower_threshold:
        send_alert(ticker, price, lower_threshold, 'down')
    
    time.sleep(3600)  # Check hourly